In [228]:
# Import libraries
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 256
batch_size = 128
max_iters = 30000
# eval_interval = 2500
learning_rate = 7e-4
eval_iters = 2500

cuda


In [229]:
# Read data
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [230]:
# Get all unique characters
chars = sorted(set(text))
vocab_size = len(chars)
print(vocab_size)
print(chars)

81
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


Level of tokenizer
- character
- word
- multi language

*More tokenizer less amount to encode and decode*

In [231]:
# Encode dict
string_to_int = {ch:i for i,ch in enumerate(chars)}
# Decode dict
int_to_string = {i:ch for i,ch in enumerate(chars)}

# Define lambda function
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

# Test
print(encode('hello world!'))
print(decode([61, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]))

[61, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]
hello world!


In [232]:
# Convert text to tensor
data = torch.tensor(encode(text),dtype = torch.long)
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [238]:
# Split train test
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

print(n)
x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

185847
inputs:
tensor([[ 9,  1, 54,  ..., 54, 73,  1],
        [61,  1, 54,  ..., 54, 71, 73],
        [58, 71, 65,  ..., 67, 58, 57],
        ...,
        [66, 54, 71,  ..., 61, 54, 73],
        [58,  1, 59,  ..., 72, 62, 65],
        [75, 62, 72,  ..., 68, 73,  1]], device='cuda:0')
targets:
tensor([[ 1, 54, 67,  ..., 73,  1, 73],
        [ 1, 54, 67,  ..., 71, 73, 61],
        [71, 65, 78,  ..., 58, 57,  1],
        ...,
        [54, 71, 64,  ..., 54, 73,  1],
        [ 1, 59, 68,  ..., 62, 65, 58],
        [62, 72, 62,  ..., 73,  1, 61]], device='cuda:0')


In [234]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [235]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


F9H
ALofyO?qZ ﻿"H,v_xdZ5I3tYG:4r1V&db c':9,p6
*xORzrIIR)ZiU!6TTN'-IRy4yXK82!3zeRyo;Ivs1!tL(F2a.rQoN12d2R?9VHDxS7J*BcWUq?QzdepEwQ3SyjOiboT9Fw_5;a:*vNBSpM_SkMllp]'gKD]lruuTHUcjXw'ozW
'jF!BK:,Q16Cq:Y0y2Bc!TJIHM8]5Kvrz3NVXX6bbt
)moTeE7.W4FzmP1j?9&d3n,"2RBz7d "2du1p[(,5EJaeZ(k8)uhP!IR[gX nQo&i8RYf)_R2RhyiPyKT8Y7X."VO-eCEeD6Abp A1gl﻿zC]JNVF9'2RG﻿ui﻿zHs2dp-:v]57dCzC2!_]Jkt 9P3.9Ce7.-k,WgeAf58)_Ss8wgguN]*b.KpMqlsO8[iUD.9qFd.F '_ cDT:9q[kLxa]x4a67d,n0q7w'3x
7)ufMbK(﻿﻿TtAC ;X]lS_t9P]lN[)Ts1C:LLU﻿iwb ;,0w4


In [236]:
# create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.844, val loss: 4.844
step: 2500, train loss: 3.073, val loss: 3.095
step: 5000, train loss: 2.551, val loss: 2.592
step: 7500, train loss: 2.451, val loss: 2.498
step: 10000, train loss: 2.430, val loss: 2.480
step: 12500, train loss: 2.424, val loss: 2.473
step: 15000, train loss: 2.420, val loss: 2.472
step: 17500, train loss: 2.419, val loss: 2.472
step: 20000, train loss: 2.419, val loss: 2.473
step: 22500, train loss: 2.419, val loss: 2.474
step: 25000, train loss: 2.418, val loss: 2.474
step: 27500, train loss: 2.419, val loss: 2.475
2.398193836212158


In [244]:
context = torch.zeros((1,1),dtype=torch.long, device=device)
print('context: ',context)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

context:  tensor([[0]], device='cuda:0')


"Yerekel Pe sale thed lorerdre t therie s fu, TE tr adeianghe hty "Winad
"Sollome.

contowanean t patrso, g tth?"
eyrintecid arod t h tucadrevep sthe otthat by holld alat wand ly tartar
wot, and dir,"

ald t foutanaie ir ngat, s te " m

Thed f ed ayoo at m, ithed
geracand
d yagoula we pan uryore thu bun_, s harest t o uglitfofloupa "
an, asapr'tow he, as. de ale sasino s thigastan  VI f arsise aizato tise  POOYerirud. ange th  k:
wld oy ne nttun intr there,
nd Itthet
sor st y thincs aroshnqu hi
